#### Week 3 Data Science Assignment : Question 3
##### Load Libraries

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# Web scrapping
from bs4 import BeautifulSoup
import urllib.request
import csv

print('Libraries imported.')

Libraries imported.


##### Functions used in cleaning and formatting data

In [5]:
#Function to convert lattitude/longitude from degree to float 
def conversion(old):
    direction = {'N':1, 'S':-1, 'E': 1, 'W':-1}
    new = old.replace(u'°',' ').replace('′',' ').replace('″',' ')
    new = new.split()
    new_dir = new.pop()
    new.extend([0,0,0])
    return (float(new[0])+float(new[1])/60.0+float(new[2])/3600.0) * direction[new_dir]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## Scrap data from  Wikipedia page'List of postal codes of Canada: M'

In [6]:
'''
# define the dataframe columns
column_names = ['Neighborhood', 'Latitude', 'Longitude'] #['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
boroughs = pd.DataFrame(columns=column_names)
boroughs

'''
print('')

###### Define the neighbourhood datarame

In [7]:
# define the dataframe columns
column_names = ['Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,Neighborhood,Latitude,Longitude


###### Specify the url to get data

In [8]:
# specify the url
urlpage =  'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
urlpage

'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

###### Get the page and parse it using BeautifulSoup

In [9]:
# query the website and return the html to the variable 'page'
page = urllib.request.urlopen(urlpage)
# parse the html using beautiful soup and store in variable 'soup'
soup = BeautifulSoup(page, 'html.parser')
#soup

###### Get the rows of the table containing data about neighbourhood

In [10]:
brhtbl = soup.find('table', class_='wikitable sortable')
trs = brhtbl.find_all('tr')
ln = len(trs)
ln

290

###### Collect data and store in the dataframe

In [11]:
for tr in trs:
    tds = tr.find_all('td')
    ltd = len(tds)
    if ltd == 0:
        continue
    pc = tds[0].getText().replace('\n','')
    br = tds[1].getText().replace('\n','')
    nh = tds[2].getText().replace('\n','')
    #print(pc,br,nh, tds[2])
    
    if 'Not assigned' in br:
        continue
    if 'Not assigned' in nh:
        nh = br
        #print (br, nh)
        nhlnksp = tds[1].find('a') #.get('href')
    else :
        nhlnksp = tds[2].find('a') #.get('href')
       
    if nhlnksp is None:
        #print(pc,br,nh, "No Link")
        continue;
    
    nhlnk = nhlnksp.get('href')
    
    p2 =  urllib.request.urlopen('https://en.wikipedia.org'+nhlnk)
    s2 = BeautifulSoup(p2, 'html.parser')
    latt='N/A'
    lng='N/A'
    em = ''
    try:
        latt = conversion(s2.find('span', attrs={'class':'latitude'}).getText())
        lng = conversion(s2.find('span', attrs={'class':'longitude'}).getText())
        #'''
        neighborhoods = neighborhoods.append({'Neighborhood': nh,
                                      'Latitude': latt,
                                      'Longitude': lng}, ignore_index=True)
        #'''
        #print(pc,br,nh, latt, lng)
    except:
        em = "Error getting lat/long"
    

    '''
    #print(pc,br,nh)
    print(br, boroughs['Borough'] == br)
    try:
        ind = boroughs[boroughs['Borough'] == br ].index
        ts = boroughs[boroughs['Borough'] == br ]['Neighborhood']
        boroughs.loc[boroughs['Borough'] == br, ['Neighborhood']] = ts[0]+','+nh
        print(br,ind,ts)
    except:
        boroughs=boroughs.append({'PostalCode':pc,'Borough':br, 'Neighborhood':nh}, ignore_index=True)
    boroughs=boroughs.append({'PostalCode':pc,'Borough':br, 'Neighborhood':nh}, ignore_index=True)
    '''
   
neighborhoods.head(10)
#print(boroughs)

,Neighborhood,Latitude,Longitude
0,Parkwoods,43.751111,-79.323056
1,Victoria Village,43.735556,-79.312500
2,Harbourfront,43.638056,-79.385000
3,Regent Park,43.661667,-79.365000
4,Lawrence Heights,43.718889,-79.450000
5,Lawrence Manor,43.725000,-79.436944
6,Queen's Park,43.664722,-79.392500
7,Rouge,43.820833,-79.206111
8,Malvern,43.811667,-79.231111
9,Woodbine Gardens,43.705556,-79.312778


In [12]:
neighborhoods.shape

(132, 3)

Use geopy library to get the latitude and longitude values of Toronto City.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>tt_explorer</em>, as shown below.

In [13]:
address = 'Toronto,ON,Canada'

geolocator = Nominatim(user_agent="tt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


##### create map of Toront city and its neighbourhoods using latitude and longitude values

In [14]:
# create map of Toront city and its neighbourhoods using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood']):
    #print(neighborhood, lat,lng)
    label = neighborhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#ff9999',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [15]:
CLIENT_ID = '2D2A1WPTXYQD0FPAL2RN3I2FSUZR5JMKOQYOPOHGYBP12NMH' # Foursquare ID
CLIENT_SECRET = '1V1RJ4ANANHR1MV3WQM4NP124KHV455HWJS5TSGVBPOBIE1W' # Foursquare Secret
VERSION = '20180605' # Foursquare API version
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

CLIENT_ID: 2D2A1WPTXYQD0FPAL2RN3I2FSUZR5JMKOQYOPOHGYBP12NMH
CLIENT_SECRET:1V1RJ4ANANHR1MV3WQM4NP124KHV455HWJS5TSGVBPOBIE1W


In [16]:
neighborhoods.loc[0, 'Neighborhood']

'Parkwoods'

Get the neighborhood's latitude and longitude values.

In [17]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.75111111111111, -79.32305555555556.


#### Now, let's get the top 100 venues that are in Parkwoods within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [18]:
radius=500
LIMIT=100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=2D2A1WPTXYQD0FPAL2RN3I2FSUZR5JMKOQYOPOHGYBP12NMH&client_secret=1V1RJ4ANANHR1MV3WQM4NP124KHV455HWJS5TSGVBPOBIE1W&ll=43.75111111111111,-79.32305555555556&v=20180605&radius=500&limit=100'

Send the GET request and examine the resutls

In [19]:
results = requests.get(url).json()
#results

In [20]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,TTC stop #8380,Bus Stop,43.752672,-79.326351
1,TTC Stop #8381,Bus Stop,43.748410,-79.326270
2,Mrs. Claus' Sweatshop,Public Art,43.753800,-79.319582
3,Broadlands Park,Park,43.746746,-79.322502
4,Broadlands Skating Rink,Skating Rink,43.746689,-79.322678


In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


## Explore Neighborhoods in Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

Since we will be using top 10 venues for clusturing, the function  not add neighbourhoods that have less than 10 venues

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    i = 0;
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        #print(name,'\t', url)
        # make the GET request
        i = i + 1
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']

            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
        except:
            print('') #print(name, "Error in getting json", i)
            #del venues_list[-1]
               
        #print(name, len(venues_list[i-1]), i)
        
        if len(venues_list[-1]) < 10:
            #print(name,  len(venues_list[-1]), i)
            del venues_list[-1]
            
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [23]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude'],
                                   radius=1000
                                  )


Let's check how many venues were returned for each neighborhood

In [24]:
toronto_venues.groupby('Neighborhood').count().shape

(116, 6)

In [25]:
print(toronto_venues.shape)
toronto_venues.head()

(6013, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.751111,-79.323056,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.751111,-79.323056,Tim Hortons,43.752814,-79.314067,Coffee Shop
2,Parkwoods,43.751111,-79.323056,Brookbanks Park,43.751976,-79.332140,Park
3,Parkwoods,43.751111,-79.323056,Bruno's valu-mart,43.746086,-79.324978,Grocery Store
4,Parkwoods,43.751111,-79.323056,Rexall,43.748493,-79.311802,Pharmacy


#### Let's find out how many unique categories can be curated from all the returned venues

In [26]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 339 uniques categories.


## Analyze Each Neighborhood

In [27]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Library,Light Rail Station,Lighthouse,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skatin

And let's examine the new dataframe size.

In [28]:
toronto_onehot.shape

(6013, 339)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [29]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Zoo,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Library,Light Rail Station,Lighthouse,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skatin

#### Let's confirm the new size

In [30]:
toronto_grouped.shape

(116, 339)

#### Let's print each neighborhood along with the top 5 most common venues

In [31]:
'''
num_top_venues = 5
 
for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
'''

'\nnum_top_venues = 5\n \nfor hood in toronto_grouped[\'Neighborhood\']:\n    print("----"+hood+"----")\n    temp = toronto_grouped[toronto_grouped[\'Neighborhood\'] == hood].T.reset_index()\n    temp.columns = [\'venue\',\'freq\']\n    temp = temp.iloc[1:]\n    temp[\'freq\'] = temp[\'freq\'].astype(float)\n    temp = temp.round({\'freq\': 2})\n    print(temp.sort_values(\'freq\', ascending=False).reset_index(drop=True).head(num_top_venues))\n    print(\'\n\')\n'

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Café,Coffee Shop,Bar,Hotel,Pub,Japanese Restaurant,Chinese Restaurant,Italian Restaurant,Asian Restaurant,Wine Bar
1,Agincourt,Shopping Mall,Chinese Restaurant,Asian Restaurant,Cantonese Restaurant,Park,Coffee Shop,Rental Car Location,Beer Store,Butcher,Restaurant
2,Agincourt North,Shopping Mall,Chinese Restaurant,Asian Restaurant,Cantonese Restaurant,Park,Coffee Shop,Rental Car Location,Beer Store,Butcher,Restaurant
3,Alderwood,Pharmacy,Hotel,Pizza Place,Pub,Discount Store,Donut Shop,Park,Skating Rink,Sandwich Place,Grocery Store
4,Bathurst Manor,Park,Middle Eastern Restaurant,Baseball Field,Arts & Crafts Store,Sandwich Place,Grocery Store,Men's Store,Hardware Store,Pharmacy,Breakfast Spot


## Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 3 clusters. $ and above didn't give better clusters

In [34]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 1, 1, 1, 2, 0, 0, 0], dtype=int32)

In [35]:
#neighborhoods_venues_sorted.drop(columns=['Cluster Labels'], inplace=True)
#neighborhoods_venues_sorted.columns

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [36]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,43.751111,-79.323056,1.0,Pharmacy,Park,Food & Drink Shop,Health Food Store,Medical Center,Bank,Tennis Court,Grocery Store,Caribbean Restaurant,Restaurant
1,Victoria Village,43.735556,-79.312500,0.0,Mediterranean Restaurant,Middle Eastern Restaurant,Grocery Store,Bubble Tea Shop,Toy / Game Store,Deli / Bodega,Asian Restaurant,Rental Car Location,Fast Food Restaurant,Gym / Fitness Center
2,Harbourfront,43.638056,-79.385000,0.0,Coffee Shop,Hotel,Café,Italian Restaurant,Aquarium,Scenic Lookout,Park,Japanese Restaurant,Brewery,Pizza Place
3,Regent Park,43.661667,-79.365000,0.0,Coffee Shop,Café,Restaurant,Park,Pub,Bakery,Thai Restaurant,Breakfast Spot,Italian Restaurant,Diner
4,Lawrence Heights,43.718889,-79.450000,0.0,Clothing Store,Cosmetics Shop,Coffee Shop,Fast Food Restaurant,Dessert Shop,Discount Store,Restaurant,Gym,Furniture / Home Store,Chocolate Shop


In [37]:
#toronto_merged['Cluster Labels'].isna().value_counts()
#remove neighbourhoods that have NA
toronto_merged = toronto_merged[np.isfinite(toronto_merged['Cluster Labels'])]

In [38]:
toronto_merged.isna().describe()

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
count,117,117,117,117,117,117,117,117,117,117,117,117,117,117
unique,1,1,1,1,1,1,1,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False,False,False,False,False,False,False,False
freq,117,117,117,117,117,117,117,117,117,117,117,117,117,117


In [39]:
toronto_merged[['Cluster Labels']] = toronto_merged[['Cluster Labels']].astype('int32')
toronto_merged.head()

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,43.751111,-79.323056,1,Pharmacy,Park,Food & Drink Shop,Health Food Store,Medical Center,Bank,Tennis Court,Grocery Store,Caribbean Restaurant,Restaurant
1,Victoria Village,43.735556,-79.312500,0,Mediterranean Restaurant,Middle Eastern Restaurant,Grocery Store,Bubble Tea Shop,Toy / Game Store,Deli / Bodega,Asian Restaurant,Rental Car Location,Fast Food Restaurant,Gym / Fitness Center
2,Harbourfront,43.638056,-79.385000,0,Coffee Shop,Hotel,Café,Italian Restaurant,Aquarium,Scenic Lookout,Park,Japanese Restaurant,Brewery,Pizza Place
3,Regent Park,43.661667,-79.365000,0,Coffee Shop,Café,Restaurant,Park,Pub,Bakery,Thai Restaurant,Breakfast Spot,Italian Restaurant,Diner
4,Lawrence Heights,43.718889,-79.450000,0,Clothing Store,Cosmetics Shop,Coffee Shop,Fast Food Restaurant,Dessert Shop,Discount Store,Restaurant,Gym,Furniture / Home Store,Chocolate Shop


In [40]:
toronto_merged['Cluster Labels'].value_counts()

0    83
1    27
2     7
Name: Cluster Labels, dtype: int64

##### Finally, let's visualize the resulting clusters

In [41]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters



In [42]:
tm1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0] + list(range(4, toronto_merged.shape[1]))]]
tm1['1st Most Common Venue'].value_counts()

Coffee Shop                   33
Café                          10
Hotel                          8
Italian Restaurant             3
Fast Food Restaurant           3
Korean Restaurant              3
Japanese Restaurant            2
Construction & Landscaping     2
Pub                            2
Harbor / Marina                2
Grocery Store                  2
Middle Eastern Restaurant      2
Vietnamese Restaurant          2
Shopping Mall                  2
Furniture / Home Store         1
Chinese Restaurant             1
Mediterranean Restaurant       1
Athletics & Sports             1
Clothing Store                 1
Pizza Place                    1
Restaurant                     1
Name: 1st Most Common Venue, dtype: int64

### As can be seen above in the 1st cluster top most 1st most common venue is 'Coffee Shop' and a lot of international cuisine restaurant so this cluster can be termed as 'Coffee Shop & international Cuisine' neighbourhood

In [43]:
tm2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0] + list(range(4, toronto_merged.shape[1]))]]
tm2['1st Most Common Venue'].value_counts()

Park                  14
Pizza Place            5
Coffee Shop            3
Pharmacy               2
Dog Run                1
Chinese Restaurant     1
Bakery                 1
Name: 1st Most Common Venue, dtype: int64

### As can be seen above in the 2nd cluster 1st most common venue are 'Park' so this cluster can be termed as 'Park' neighbourhood

In [44]:
tm3 =toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0] + list(range(4, toronto_merged.shape[1]))]]
tm3['1st Most Common Venue'].value_counts()

Fast Food Restaurant    3
Indian Restaurant       3
Coffee Shop             1
Name: 1st Most Common Venue, dtype: int64

### As can be seen above in the 3rd cluster majority of 1st most common venue are different kinds of restaurants so this cluster can be termed as 'Eating Out' neighbourhoods